In [1]:
import pickle
import os
import re
import numpy as np
from gensim.models import word2vec
import logging
import pandas as pd

In [2]:
import sys
sys.path.insert(0, '../Core-scripts/')

from parse_and_prepare import ProteinProteinInteractionClassifier as ppi
import file_readers as fr
import prediction as pred

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [7]:
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [4]:
yeast_strict_real = pickle.load(open('../../Results/Yeast/yeast_mentions_strict_real.pkl', 'rb'))
yeast_gen_real = pickle.load(open('../../Results/Yeast/yeast_mentions_gen_real.pkl', 'rb'))
yeast_be_real = pickle.load(open('../../Results/Yeast/yeast_mentions_be_real.pkl', 'rb'))
random_seeds = [144, 235, 905, 2895, 3462, 4225, 5056, 5192, 7751, 7813]

In [5]:
for seed in random_seeds:
    real_tr_te_name = 'Yeast/train_test/yeast_tr_te_split_' + str(seed)
    train_data, b, c, d = pred.manual_train_test_split(yeast_strict_real, real_tr_te_name, random_state=seed ,test_set_prop=0.1)

In [6]:
yeast_w2v_model_strict = pred.make_w2v_model(yeast_strict_real, 'Yeast/models/yeast_strict')
yeast_w2v_model_gen = pred.make_w2v_model(yeast_gen_real, 'Yeast/models/yeast_gen')
yeast_w2v_model_be = pred.make_w2v_model(yeast_be_real, 'Yeast/models/yeast_be')

2017-05-30 08:53:56,773 : INFO : collecting all words and their counts
2017-05-30 08:53:56,774 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-30 08:53:56,817 : INFO : collected 11431 word types from a corpus of 188505 raw words and 7145 sentences
2017-05-30 08:53:56,819 : INFO : Loading a fresh vocabulary
2017-05-30 08:53:56,835 : INFO : min_count=8 retains 2372 unique words (20% of original 11431, drops 9059)
2017-05-30 08:53:56,837 : INFO : min_count=8 leaves 169689 word corpus (90% of original 188505, drops 18816)
2017-05-30 08:53:56,845 : INFO : deleting the raw counts dictionary of 11431 items
2017-05-30 08:53:56,846 : INFO : sample=0.001 downsamples 43 most-common words
2017-05-30 08:53:56,847 : INFO : downsampling leaves estimated 119416 word corpus (70.4% of prior 169689)
2017-05-30 08:53:56,847 : INFO : estimated required memory for 2372 words and 800 dimensions: 24431600 bytes
2017-05-30 08:53:56,856 : INFO : constructing a huffman tree fro

Parsing datasets sentences
Training Word2Vec Model


2017-05-30 08:53:56,972 : INFO : training model with 4 workers on 2372 vocabulary and 800 features, using sg=1 hs=1 sample=0.001 negative=5 window=8
2017-05-30 08:53:56,974 : INFO : expecting 7145 sentences, matching count from corpus used for vocabulary survey
2017-05-30 08:53:58,034 : INFO : PROGRESS: at 7.44% examples, 42053 words/s, in_qsize 8, out_qsize 1
2017-05-30 08:53:59,072 : INFO : PROGRESS: at 19.06% examples, 54436 words/s, in_qsize 7, out_qsize 0
2017-05-30 08:54:00,115 : INFO : PROGRESS: at 29.64% examples, 56480 words/s, in_qsize 7, out_qsize 0
2017-05-30 08:54:01,339 : INFO : PROGRESS: at 41.32% examples, 56583 words/s, in_qsize 7, out_qsize 0
2017-05-30 08:54:02,500 : INFO : PROGRESS: at 53.96% examples, 58434 words/s, in_qsize 7, out_qsize 0
2017-05-30 08:54:03,695 : INFO : PROGRESS: at 66.79% examples, 59339 words/s, in_qsize 7, out_qsize 0
2017-05-30 08:54:04,819 : INFO : PROGRESS: at 79.46% examples, 60495 words/s, in_qsize 7, out_qsize 0
2017-05-30 08:54:05,842 :

Parsing datasets sentences


2017-05-30 08:54:07,518 : INFO : collecting all words and their counts
2017-05-30 08:54:07,519 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-30 08:54:07,562 : INFO : PROGRESS: at sentence #10000, processed 242489 words, keeping 11178 word types
2017-05-30 08:54:07,604 : INFO : PROGRESS: at sentence #20000, processed 484360 words, keeping 14874 word types
2017-05-30 08:54:07,647 : INFO : PROGRESS: at sentence #30000, processed 727638 words, keeping 17573 word types
2017-05-30 08:54:07,691 : INFO : PROGRESS: at sentence #40000, processed 970418 words, keeping 19777 word types


...

2017-05-30 08:54:07,725 : INFO : collected 21147 word types from a corpus of 1155938 raw words and 47662 sentences
2017-05-30 08:54:07,726 : INFO : Loading a fresh vocabulary
2017-05-30 08:54:07,914 : INFO : min_count=8 retains 6303 unique words (29% of original 21147, drops 14844)
2017-05-30 08:54:07,915 : INFO : min_count=8 leaves 1123722 word corpus (97% of original 1155938, drops 32216)
2017-05-30 08:54:07,930 : INFO : deleting the raw counts dictionary of 21147 items
2017-05-30 08:54:07,931 : INFO : sample=0.001 downsamples 48 most-common words
2017-05-30 08:54:07,932 : INFO : downsampling leaves estimated 817860 word corpus (72.8% of prior 1123722)
2017-05-30 08:54:07,933 : INFO : estimated required memory for 6303 words and 800 dimensions: 64920900 bytes
2017-05-30 08:54:07,944 : INFO : constructing a huffman tree from 6303 words
2017-05-30 08:54:08,076 : INFO : built huffman tree with maximum node depth 17
2017-05-30 08:54:08,088 : INFO : resetting layer weights
2017-05-30 08:5

Parsing datasets sentences


2017-05-30 08:55:46,119 : INFO : collecting all words and their counts
2017-05-30 08:55:46,120 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-30 08:55:46,173 : INFO : PROGRESS: at sentence #10000, processed 229206 words, keeping 12313 word types
2017-05-30 08:55:46,226 : INFO : PROGRESS: at sentence #20000, processed 457084 words, keeping 16840 word types
2017-05-30 08:55:46,278 : INFO : PROGRESS: at sentence #30000, processed 686842 words, keeping 20179 word types


Training Word2Vec Model


2017-05-30 08:55:46,332 : INFO : PROGRESS: at sentence #40000, processed 915527 words, keeping 22886 word types
2017-05-30 08:55:46,395 : INFO : PROGRESS: at sentence #50000, processed 1144562 words, keeping 25201 word types
2017-05-30 08:55:46,447 : INFO : PROGRESS: at sentence #60000, processed 1374108 words, keeping 27166 word types
2017-05-30 08:55:46,502 : INFO : PROGRESS: at sentence #70000, processed 1602041 words, keeping 28983 word types
2017-05-30 08:55:46,557 : INFO : PROGRESS: at sentence #80000, processed 1832714 words, keeping 30757 word types
2017-05-30 08:55:46,612 : INFO : PROGRESS: at sentence #90000, processed 2060897 words, keeping 32306 word types
2017-05-30 08:55:46,667 : INFO : PROGRESS: at sentence #100000, processed 2289067 words, keeping 33725 word types
2017-05-30 08:55:46,721 : INFO : PROGRESS: at sentence #110000, processed 2516540 words, keeping 35030 word types
2017-05-30 08:55:46,776 : INFO : PROGRESS: at sentence #120000, processed 2745541 words, keepin

In [9]:
for seed in random_seeds:
    data_name = '../../Results/Yeast/train_test/yeast_tr_te_split_' + str(seed)
    train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
    train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
    test_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
    test_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))

    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_strict, feature_count=800)

    strict_list_SR_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                 train_labels, test_labels]
    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_gen, feature_count=800)

    strict_list_GEN_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                  train_labels, test_labels]

    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_be, feature_count=800)

    strict_list_BE_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                 train_labels, test_labels]

    pickle.dump(strict_list_SR_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'wb'))
    pickle.dump(strict_list_GEN_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'wb'))
    pickle.dump(strict_list_BE_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'wb'))

../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


In [10]:
def modelfit(alg, train_vecs, train_labels, w2v_model_type, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param=alg.get_xgb_params()
        xgtrain = xgb.DMatrix(train_vecs, 
                              label=train_labels)
        cvresult = xgb.cv(xgb_param, 
                          xgtrain, 
                          num_boost_round=alg.get_params()['n_estimators'], 
                          nfold=cv_folds, 
                          metrics='auc', 
                          early_stopping_rounds=50)
        alg.set_params(n_estimators=cvresult.shape[0])
        
    #fit the algorithm on the data
    alg.fit(train_vecs, train_labels, eval_metric='auc')
    
    #Predict training set:
    train_predictions = alg.predict(train_vecs)
    train_predprob = alg.predict_proba(train_vecs)[:,1]
    
    #Print Model report:
    print(w2v_model_type, '\nModel Report')
    print(w2v_model_type, 'Accuracy: %.4g' % metrics.accuracy_score(train_labels, train_predictions))
    print(w2v_model_type, 'AUC Score (Train): %f' % metrics.roc_auc_score(train_labels, train_predprob))
    
#     feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importance')
#     plt.ylabel('Feature Importance Score')

    error = 1-metrics.accuracy_score(train_labels, train_predictions)
    auc = metrics.roc_auc_score(train_labels, train_predprob)
    
    return error, auc

In [7]:
xgb1 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=5, 
                     min_child_weight=1, 
                     gamma=0, 
                     subsample=0.8, 
                     colsample_bytree=0.8, 
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

strict_error_list = []
strict_auc_list = []
gen_error_list = []
gen_auc_list = []
be_error_list = []
be_auc_list = []


for seed in random_seeds:  
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    print(seed)
    strict_error, strict_auc = modelfit(xgb1, strict_list[0], strict_list[2], 'STRICT')
    gen_error, gen_auc = modelfit(xgb1, gen_list[0], gen_list[2], 'GEN')
    be_error, be_auc = modelfit(xgb1, be_list[0], be_list[2], 'BE')
    
    strict_error_list.append(strict_error)
    strict_auc_list.append(strict_auc)
    gen_error_list.append(gen_error)
    gen_auc_list.append(gen_auc)
    be_error_list.append(be_error)
    be_auc_list.append(be_auc)

144
STRICT 
Model Report
STRICT Accuracy: 0.6086
STRICT AUC Score (Train): 0.607743
GEN 
Model Report
GEN Accuracy: 0.5843
GEN AUC Score (Train): 0.581177
BE 
Model Report
BE Accuracy: 0.5965
BE AUC Score (Train): 0.598749
235
STRICT 
Model Report
STRICT Accuracy: 0.6004
STRICT AUC Score (Train): 0.614761
GEN 
Model Report
GEN Accuracy: 0.6044
GEN AUC Score (Train): 0.599407
BE 
Model Report
BE Accuracy: 0.6018
BE AUC Score (Train): 0.592301
905
STRICT 
Model Report
STRICT Accuracy: 0.6131
STRICT AUC Score (Train): 0.627748
GEN 
Model Report
GEN Accuracy: 0.5965
GEN AUC Score (Train): 0.603445
BE 
Model Report
BE Accuracy: 0.5879
BE AUC Score (Train): 0.571791
2895
STRICT 
Model Report
STRICT Accuracy: 0.5841
STRICT AUC Score (Train): 0.587349
GEN 
Model Report
GEN Accuracy: 0.5781
GEN AUC Score (Train): 0.560142
BE 
Model Report
BE Accuracy: 0.5917
BE AUC Score (Train): 0.584521
3462
STRICT 
Model Report
STRICT Accuracy: 0.5953
STRICT AUC Score (Train): 0.613894
GEN 
Model Report
GEN 

In [9]:
xgb1 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=5, 
                     min_child_weight=1, 
                     gamma=0, 
                     subsample=0.8, 
                     colsample_bytree=0.8, 
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

strict_error_list = []
strict_auc_list = []
gen_error_list = []
gen_auc_list = []
be_error_list = []
be_auc_list = []


for seed in random_seeds:  
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    print(seed)
    strict_error, strict_auc = modelfit(xgb1, strict_list[0], strict_list[2], 'STRICT')
    gen_error, gen_auc = modelfit(xgb1, gen_list[0], gen_list[2], 'GEN')
    be_error, be_auc = modelfit(xgb1, be_list[0], be_list[2], 'BE')
    
    strict_error_list.append(strict_error)
    strict_auc_list.append(strict_auc)
    gen_error_list.append(gen_error)
    gen_auc_list.append(gen_auc)
    be_error_list.append(be_error)
    be_auc_list.append(be_auc)

144
STRICT 
Model Report
STRICT Accuracy: 0.637
STRICT AUC Score (Train): 0.737125
GEN 
Model Report
GEN Accuracy: 0.6552
GEN AUC Score (Train): 0.748667
BE 
Model Report
BE Accuracy: 0.6712
BE AUC Score (Train): 0.785685
235
STRICT 
Model Report
STRICT Accuracy: 0.6624
STRICT AUC Score (Train): 0.739747
GEN 
Model Report
GEN Accuracy: 0.6504
GEN AUC Score (Train): 0.738340
BE 
Model Report
BE Accuracy: 0.6855
BE AUC Score (Train): 0.784790
905
STRICT 
Model Report
STRICT Accuracy: 0.6512
STRICT AUC Score (Train): 0.746496
GEN 
Model Report
GEN Accuracy: 0.6653
GEN AUC Score (Train): 0.765681
BE 
Model Report
BE Accuracy: 0.6767
BE AUC Score (Train): 0.790130
2895
STRICT 
Model Report
STRICT Accuracy: 0.6491
STRICT AUC Score (Train): 0.744200
GEN 
Model Report
GEN Accuracy: 0.6756
GEN AUC Score (Train): 0.776996
BE 
Model Report
BE Accuracy: 0.687
BE AUC Score (Train): 0.789860
3462
STRICT 
Model Report
STRICT Accuracy: 0.6515
STRICT AUC Score (Train): 0.742703
GEN 
Model Report
GEN Ac

In [12]:
xgb1 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=6, 
                     min_child_weight=1, 
                     gamma=0.2, 
                     subsample=0.6, 
                     colsample_bytree=0.8,
                     reg_alpha = 0.01,
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

strict_error_list = []
strict_auc_list = []
gen_error_list = []
gen_auc_list = []
be_error_list = []
be_auc_list = []


for seed in random_seeds:  
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    print(seed)
    strict_error, strict_auc = modelfit(xgb1, strict_list[0], strict_list[2], 'STRICT')
    gen_error, gen_auc = modelfit(xgb1, gen_list[0], gen_list[2], 'GEN')
    be_error, be_auc = modelfit(xgb1, be_list[0], be_list[2], 'BE')
    
    strict_error_list.append(strict_error)
    strict_auc_list.append(strict_auc)
    gen_error_list.append(gen_error)
    gen_auc_list.append(gen_auc)
    be_error_list.append(be_error)
    be_auc_list.append(be_auc)

144
STRICT 
Model Report
STRICT Accuracy: 0.8676
STRICT AUC Score (Train): 0.950801
GEN 
Model Report
GEN Accuracy: 0.8706
GEN AUC Score (Train): 0.958366
BE 
Model Report
BE Accuracy: 0.8997
BE AUC Score (Train): 0.973117
235
STRICT 
Model Report
STRICT Accuracy: 0.845
STRICT AUC Score (Train): 0.948836
GEN 
Model Report
GEN Accuracy: 0.851
GEN AUC Score (Train): 0.956173
BE 
Model Report
BE Accuracy: 0.8642
BE AUC Score (Train): 0.963154
905
STRICT 
Model Report
STRICT Accuracy: 0.8262
STRICT AUC Score (Train): 0.938901
GEN 
Model Report
GEN Accuracy: 0.8519
GEN AUC Score (Train): 0.961507
BE 
Model Report
BE Accuracy: 0.8753
BE AUC Score (Train): 0.970509
2895
STRICT 
Model Report
STRICT Accuracy: 0.8137
STRICT AUC Score (Train): 0.930456
GEN 
Model Report
GEN Accuracy: 0.8382
GEN AUC Score (Train): 0.950308
BE 
Model Report
BE Accuracy: 0.8613
BE AUC Score (Train): 0.961133
3462
STRICT 
Model Report
STRICT Accuracy: 0.8333
STRICT AUC Score (Train): 0.938910
GEN 
Model Report
GEN Ac

In [7]:
print('Strict Results Error: ', np.mean(strict_error_list), ', Auc: ', np.mean(strict_auc_list))
print('Gen Results Error: ', np.mean(gen_error_list), ', Auc: ', np.mean(gen_auc_list))
print('Be Results Error: ', np.mean(be_error_list), ', Auc: ', np.mean(be_auc_list))

Strict Results Error:  0.330430479588 , Auc:  0.768074209551
Gen Results Error:  0.364579946523 , Auc:  0.707197019221
Be Results Error:  0.344158176436 , Auc:  0.74322406605


In [13]:
print('Strict Results Error: ', np.mean(strict_error_list), ', Auc: ', np.mean(strict_auc_list))
print('Gen Results Error: ', np.mean(gen_error_list), ', Auc: ', np.mean(gen_auc_list))
print('Be Results Error: ', np.mean(be_error_list), ', Auc: ', np.mean(be_auc_list))

Strict Results Error:  0.165030797764 , Auc:  0.94070425964
Gen Results Error:  0.14109773617 , Auc:  0.957793160646
Be Results Error:  0.123702592979 , Auc:  0.967173857003


In [8]:
param_test1 = {
    'max_depth':[3,4,5,6,7,8,9,10],
    'min_child_weight':[1,2,3,4,5,6]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=140, 
                                                  max_depth=5, 
                                                  min_child_weight=1, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8, 
                                                  objective='binary:logistic', 
                                                  nthread=4, 
                                                  scale_pos_weight=1, 
                                                  seed=24), 
                        param_grid=param_test1, 
                        scoring='roc_auc',  
                        iid=False, 
                        cv=5)

In [10]:
strict_param_test1 = []
for seed in [144]:
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(strict_list[0], strict_list[2])
    strict_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

In [11]:
gen_param_test1 = []
for seed in [144]:
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(gen_list[0], gen_list[2])
    gen_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

In [12]:
be_param_test1 = []
for seed in [144]:
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(be_list[0], be_list[2])
    be_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

In [13]:
print(strict_param_test1, gen_param_test1, be_param_test1)

[(144, ({'max_depth': 3, 'min_child_weight': 1}, 0.5184627231701338))] [(144, ({'max_depth': 3, 'min_child_weight': 5}, 0.5033950980591407))] [(144, ({'max_depth': 6, 'min_child_weight': 6}, 0.4984421837963776))]


In [14]:
xgb2 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=3, 
                     min_child_weight=3, 
                     gamma=0, 
                     subsample=0.8, 
                     colsample_bytree=0.8, 
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

strict_error_list = []
strict_auc_list = []
gen_error_list = []
gen_auc_list = []
be_error_list = []
be_auc_list = []


for seed in random_seeds:  
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    print(seed)
    strict_error, strict_auc = modelfit(xgb2, strict_list[0], strict_list[2], 'STRICT')
    gen_error, gen_auc = modelfit(xgb2, gen_list[0], gen_list[2], 'GEN')
    be_error, be_auc = modelfit(xgb2, be_list[0], be_list[2], 'BE')
    
    strict_error_list.append(strict_error)
    strict_auc_list.append(strict_auc)
    gen_error_list.append(gen_error)
    gen_auc_list.append(gen_auc)
    be_error_list.append(be_error)
    be_auc_list.append(be_auc)

144
STRICT 
Model Report
STRICT Accuracy: 0.712
STRICT AUC Score (Train): 0.869563
GEN 
Model Report
GEN Accuracy: 0.6677
GEN AUC Score (Train): 0.787012
BE 
Model Report
BE Accuracy: 0.5891
BE AUC Score (Train): 0.640011
235
STRICT 
Model Report
STRICT Accuracy: 0.581
STRICT AUC Score (Train): 0.660918
GEN 
Model Report
GEN Accuracy: 0.586
GEN AUC Score (Train): 0.627057
BE 
Model Report
BE Accuracy: 0.5887
BE AUC Score (Train): 0.651093
905
STRICT 
Model Report
STRICT Accuracy: 0.5811
STRICT AUC Score (Train): 0.656544
GEN 
Model Report
GEN Accuracy: 0.5906
GEN AUC Score (Train): 0.629023
BE 
Model Report
BE Accuracy: 0.5933
BE AUC Score (Train): 0.649648
2895
STRICT 
Model Report
STRICT Accuracy: 0.5872
STRICT AUC Score (Train): 0.674281
GEN 
Model Report
GEN Accuracy: 0.5776
GEN AUC Score (Train): 0.640244
BE 
Model Report
BE Accuracy: 0.5947
BE AUC Score (Train): 0.655433
3462
STRICT 
Model Report
STRICT Accuracy: 0.5994
STRICT AUC Score (Train): 0.673769
GEN 
Model Report
GEN Acc